# LOGIT MODEL을 활용한 Feature selection

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'malgun gothic'
from statsmodels.formula.api import logit

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
df_full = pd.read_csv('..\..\..\Code\SH\재무변수_특수문자제외_NaN.csv', index_col = 0)

In [9]:
df_full

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계_요약_,유동자산_요약_,당좌자산_요약_,현금_및_현금성자산_요약_,...,기업가치_EV_,EBITDA,EBITDA_매출액,EBITDA_금융비용,EBITDA_평균발행주식수,EV_EBITDA,년,key,공시제목,부도
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2001.0,588202001,NaN,1.0
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,588202002,NaN,1.0
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,588202003,NaN,1.0
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,588202004,NaN,1.0
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2005.0,588202005,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,5.0,NaN,UQ,79137.0,14794.0,12274.0,1440.0,...,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,NaN,1.0
27353,흥구석유(주),24060.0,2017/12,5.0,NaN,UQ,79912.0,15880.0,13124.0,1588.0,...,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,NaN,1.0
27354,흥구석유(주),24060.0,2018/12,5.0,NaN,UQ,75206.0,12515.0,11029.0,4017.0,...,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,NaN,1.0
27355,흥구석유(주),24060.0,2019/12,5.0,NaN,UQ,83661.0,22620.0,19701.0,2318.0,...,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,NaN,1.0


In [10]:
df = df_full.iloc[:,6:267]

In [11]:
df

,자산총계_요약_,유동자산_요약_,당좌자산_요약_,현금_및_현금성자산_요약_,단기투자자산_요약_,단기예금_요약_,단기매매증권_요약_,단기대여금_요약_,기타단기투자자산_요약_,매출채권_요약_,...,PCR_Price_cash_flow_ratio__최고_,PCR_Price_cash_flow_ratio__최저_,PSR_Price_sales_ratio__최고_,PSR_Price_sales_ratio__최저_,기업가치_EV_,EBITDA,EBITDA_매출액,EBITDA_금융비용,EBITDA_평균발행주식수,EV_EBITDA
0,28990.0,21169.0,10541.0,4330.0,4552.0,4539.0,1.0,13.0,0.0,786.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20908.0,16359.0,2307.0,1829.0,18.0,0.0,6.0,13.0,0.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15401.0,9880.0,1436.0,355.0,246.0,240.0,6.0,0.0,0.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11063.0,8534.0,1083.0,2.0,381.0,380.0,0.0,0.0,1.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5179.0,3893.0,445.0,11.0,0.0,0.0,0.0,0.0,0.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,79137.0,14794.0,12274.0,1440.0,5065.0,0.0,0.0,0.0,5065.0,5158.0,...,55.71,35.62,0.43,0.28,55934.79,1348.52,0.94,10.71,89.90,41.48
27353,79912.0,15880.0,13124.0,1588.0,5058.0,0.0,0.0,0.0,5058.0,6315.0,...,33.70,24.31,0.34,0.24,46036.63,1147.56,0.74,9.02,76.50,40.12
27354,75206.0,12515.0,11029.0,4017.0,5.0,0.0,0.0,0.0,5.0,4403.0,...,13.22,7.82,0.40,0.24,36557.57,1451.79,0.87,16.83,96.79,25.18
27355,83661.0,22620.0,19701.0,2318.0,11472.0,0.0,0.0,0.0,11472.0,5161.0,...,176.30,51.31,0.86,0.25,78081.78,1009.25,0.67,507.93,67.28,77.37


In [18]:
del col

In [19]:
df.columns.tolist()

['자산총계_요약_',
 '유동자산_요약_',
 '당좌자산_요약_',
 '현금_및_현금성자산_요약_',
 '단기투자자산_요약_',
 '단기예금_요약_',
 '단기매매증권_요약_',
 '단기대여금_요약_',
 '기타단기투자자산_요약_',
 '매출채권_요약_',
 '_할인어음_또는_배서어음_요약_',
 '기타당좌자산_요약_',
 '재고자산_요약_',
 '상품_요약_',
 '제품_요약_',
 '반제품_요약_',
 '재공품_요약_',
 '원재료_요약_',
 '기타재고자산_요약_',
 '임대주택자산_요약_',
 '비유동자산_요약_',
 '투자자산_요약_',
 '_장기금융상품_요약_',
 '_투자유가증권_장기투자증권__요약_',
 '유형자산_요약_',
 '토지',
 '건설중인자산',
 '건물',
 '기계장치',
 '차량운반구',
 '렌탈자산',
 '설비자산',
 '무형자산_요약_',
 '기타비유동자산_요약_',
 '이연자산_요약_',
 '부채총계_요약_',
 '유동부채_요약_',
 '매입채무_요약_',
 '단기차입금_요약_',
 '유동성장기부채_요약_',
 '기타유동부채_요약_',
 '비유동부채_요약_',
 '사채_요약_',
 '장기차입금_요약_',
 '제충당금_요약_',
 '기타비유동부채_요약_',
 '이연부채_요약_',
 '자본총계_요약_',
 '자본금_요약_',
 '자본잉여금_요약_',
 '자본조정_요약_',
 '기타포괄손익누계액_요약_',
 '이익잉여금_요약_',
 '_미처분이익잉여금_및_차기이월미처분이익잉여금_요약_',
 '_당기순이익_요약_',
 '부채와자본총계_요약_',
 '매출액_요약_',
 '매출원가_요약_',
 '매출총이익_요약_',
 '판매비와_관리비_요약_',
 '급료',
 '퇴직급여_요약_',
 '복리후생비_요약_',
 '세금과공과_요약_',
 '임차료_요약_',
 '감가상각비',
 '연구비',
 '기타_판매비와_관리비_요약_',
 '영업이익_요약_',
 '영업외수익_요약_',
 '이자수익_요약_',
 '배당금수익_요약_',
 '외환차익_요약_',

In [20]:
col = df.columns.tolist()
col.append('부도')

In [23]:
df = df_full[col]

In [25]:
df = df.fillna(0)

In [26]:
col

['자산총계_요약_',
 '유동자산_요약_',
 '당좌자산_요약_',
 '현금_및_현금성자산_요약_',
 '단기투자자산_요약_',
 '단기예금_요약_',
 '단기매매증권_요약_',
 '단기대여금_요약_',
 '기타단기투자자산_요약_',
 '매출채권_요약_',
 '_할인어음_또는_배서어음_요약_',
 '기타당좌자산_요약_',
 '재고자산_요약_',
 '상품_요약_',
 '제품_요약_',
 '반제품_요약_',
 '재공품_요약_',
 '원재료_요약_',
 '기타재고자산_요약_',
 '임대주택자산_요약_',
 '비유동자산_요약_',
 '투자자산_요약_',
 '_장기금융상품_요약_',
 '_투자유가증권_장기투자증권__요약_',
 '유형자산_요약_',
 '토지',
 '건설중인자산',
 '건물',
 '기계장치',
 '차량운반구',
 '렌탈자산',
 '설비자산',
 '무형자산_요약_',
 '기타비유동자산_요약_',
 '이연자산_요약_',
 '부채총계_요약_',
 '유동부채_요약_',
 '매입채무_요약_',
 '단기차입금_요약_',
 '유동성장기부채_요약_',
 '기타유동부채_요약_',
 '비유동부채_요약_',
 '사채_요약_',
 '장기차입금_요약_',
 '제충당금_요약_',
 '기타비유동부채_요약_',
 '이연부채_요약_',
 '자본총계_요약_',
 '자본금_요약_',
 '자본잉여금_요약_',
 '자본조정_요약_',
 '기타포괄손익누계액_요약_',
 '이익잉여금_요약_',
 '_미처분이익잉여금_및_차기이월미처분이익잉여금_요약_',
 '_당기순이익_요약_',
 '부채와자본총계_요약_',
 '매출액_요약_',
 '매출원가_요약_',
 '매출총이익_요약_',
 '판매비와_관리비_요약_',
 '급료',
 '퇴직급여_요약_',
 '복리후생비_요약_',
 '세금과공과_요약_',
 '임차료_요약_',
 '감가상각비',
 '연구비',
 '기타_판매비와_관리비_요약_',
 '영업이익_요약_',
 '영업외수익_요약_',
 '이자수익_요약_',
 '배당금수익_요약_',
 '외환차익_요약_',

In [32]:
df

,자산총계_요약_,유동자산_요약_,당좌자산_요약_,현금_및_현금성자산_요약_,단기투자자산_요약_,단기예금_요약_,단기매매증권_요약_,단기대여금_요약_,기타단기투자자산_요약_,매출채권_요약_,...,PCR_Price_cash_flow_ratio__최저_,PSR_Price_sales_ratio__최고_,PSR_Price_sales_ratio__최저_,기업가치_EV_,EBITDA,EBITDA_매출액,EBITDA_금융비용,EBITDA_평균발행주식수,EV_EBITDA,부도
0,28990.0,21169.0,10541.0,4330.0,4552.0,4539.0,1.0,13.0,0.0,786.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
1,20908.0,16359.0,2307.0,1829.0,18.0,0.0,6.0,13.0,0.0,14.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
2,15401.0,9880.0,1436.0,355.0,246.0,240.0,6.0,0.0,0.0,26.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
3,11063.0,8534.0,1083.0,2.0,381.0,380.0,0.0,0.0,1.0,72.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
4,5179.0,3893.0,445.0,11.0,0.0,0.0,0.0,0.0,0.0,33.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,79137.0,14794.0,12274.0,1440.0,5065.0,0.0,0.0,0.0,5065.0,5158.0,...,35.62,0.43,0.28,55934.79,1348.52,0.94,10.71,89.90,41.48,1.0
27353,79912.0,15880.0,13124.0,1588.0,5058.0,0.0,0.0,0.0,5058.0,6315.0,...,24.31,0.34,0.24,46036.63,1147.56,0.74,9.02,76.50,40.12,1.0
27354,75206.0,12515.0,11029.0,4017.0,5.0,0.0,0.0,0.0,5.0,4403.0,...,7.82,0.40,0.24,36557.57,1451.79,0.87,16.83,96.79,25.18,1.0
27355,83661.0,22620.0,19701.0,2318.0,11472.0,0.0,0.0,0.0,11472.0,5161.0,...,51.31,0.86,0.25,78081.78,1009.25,0.67,507.93,67.28,77.37,1.0


In [27]:
col_ = df_full.iloc[:,6:267].columns.tolist()

In [30]:
'+'.join(df.columns)

'자산총계_요약_+유동자산_요약_+당좌자산_요약_+현금_및_현금성자산_요약_+단기투자자산_요약_+단기예금_요약_+단기매매증권_요약_+단기대여금_요약_+기타단기투자자산_요약_+매출채권_요약_+_할인어음_또는_배서어음_요약_+기타당좌자산_요약_+재고자산_요약_+상품_요약_+제품_요약_+반제품_요약_+재공품_요약_+원재료_요약_+기타재고자산_요약_+임대주택자산_요약_+비유동자산_요약_+투자자산_요약_+_장기금융상품_요약_+_투자유가증권_장기투자증권__요약_+유형자산_요약_+토지+건설중인자산+건물+기계장치+차량운반구+렌탈자산+설비자산+무형자산_요약_+기타비유동자산_요약_+이연자산_요약_+부채총계_요약_+유동부채_요약_+매입채무_요약_+단기차입금_요약_+유동성장기부채_요약_+기타유동부채_요약_+비유동부채_요약_+사채_요약_+장기차입금_요약_+제충당금_요약_+기타비유동부채_요약_+이연부채_요약_+자본총계_요약_+자본금_요약_+자본잉여금_요약_+자본조정_요약_+기타포괄손익누계액_요약_+이익잉여금_요약_+_미처분이익잉여금_및_차기이월미처분이익잉여금_요약_+_당기순이익_요약_+부채와자본총계_요약_+매출액_요약_+매출원가_요약_+매출총이익_요약_+판매비와_관리비_요약_+급료+퇴직급여_요약_+복리후생비_요약_+세금과공과_요약_+임차료_요약_+감가상각비+연구비+기타_판매비와_관리비_요약_+영업이익_요약_+영업외수익_요약_+이자수익_요약_+배당금수익_요약_+외환차익_요약_+외화환산이익_요약_+지분법이익_요약_+기타영업외수익_요약_+영업외비용_요약_+이자비용_요약_+외환차손_요약_+외화환산손실_요약_+지분법손실_요약_+기타영업외비용_요약_+법인세비용차감전_계속사업_손익_요약_+_계속사업손익_법인세비용_요약_+계속사업이익_요약_+중단사업이익_요약_+_법인세효과_요약_+당기순이익_요약_+주당계속사업이익_요약_+주당순이익_요약_+희석화주당계속사업이익_요약_+희석화주당순이익_요약_+CPA수정후_당기순이익_요약_+_주당경상이익_2007년_이전_발생__요약_+_희석화주당경상이익_2007년_이전_

In [41]:
logit('부도 ~ 자산총계_요약_+유동자산_요약_+당좌자산_요약_+현금_및_현금성자산_요약_+단기투자자산_요약_+단기예금_요약_+단기매매증권_요약_+단기대여금_요약_+기타단기투자자산_요약_+매출채권_요약_+_할인어음_또는_배서어음_요약_+기타당좌자산_요약_+재고자산_요약_+상품_요약_+제품_요약_+반제품_요약_+재공품_요약_+원재료_요약_+기타재고자산_요약_+임대주택자산_요약_+비유동자산_요약_+투자자산_요약_+_장기금융상품_요약_+_투자유가증권_장기투자증권__요약_+유형자산_요약_+토지+건설중인자산+건물+기계장치+차량운반구+렌탈자산+설비자산+무형자산_요약_+기타비유동자산_요약_+이연자산_요약_+부채총계_요약_+유동부채_요약_+매입채무_요약_',
      data=df).fit().summary()

         Current function value: 0.066174
         Iterations: 35


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     부도   No. Observations:                27357
Model:                          Logit   Df Residuals:                    27318
Method:                           MLE   Df Model:                           38
Date:                Fri, 03 Dec 2021   Pseudo R-squ.:                  0.1996
Time:                        11:30:22   Log-Likelihood:                -1810.3
converged:                      False   LL-Null:                       -2261.8
Covariance Type:            nonrobust   LLR p-value:                7.875e-165
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               2.9997      0.071     42.038      0.000       2.860       3.140
자산총계_요약_               -0.0984      0.117     -0.840      0.401      -0.328       0.131
유동자산_요약_                0.0410      0.138      0.297      0.767      -0.229       0.311
당좌자산_요약_                0.0845      0.128      0.660      0.509      -0.166       0.335
현금_및_현금성자산_요약_         -0.0269      0.074     -0.365      0.715      -0.171       0.118
단기투자자산_요약_             -0.1630      0.184     -0.886      0.376      -0.524       0.198
단기예금_요약_                0.1361      0.179      0.760      0.447      -0.215       0.487
단기매매증권_요약_              0.1363      0.179      0.761      0.447      -0.215       0.488
단기대여금_요약_               0.1359      0.179      0.759      0.448      -0.215       0.487
기타단기투자자산_요약_            0.1360      0.179      0.759      0.448      -0.215       0.487
매출채권_요약_               -0.0270      0.074     -0.366      0.714      -0.171       0.117
_할인어음_또는_배서어음_요약_       0.0001   8.16e-05      1.838      0.066   -9.91e-06       0.000
기타당좌자산_요약_             -0.0270      0.074     -0.366      0.714      -0.171       0.117
재고자산_요약_                0.0605      0.123      0.490      0.624      -0.181       0.302
상품_요약_                 -0.0028      0.019     -0.150      0.881      -0.040       0.034
제품_요약_                 -0.0029      0.019     -0.156      0.876      -0.040       0.034
반제품_요약_                -0.0026      0.019     -0.138      0.891      -0.039       0.034
재공품_요약_                -0.0028      0.019     -0.152      0.880      -0.040       0.034
원재료_요약_                -0.0030      0.019     -0.159      0.874      -0.040       0.034
기타재고자산_요약_             -0.0030      0.019     -0.160      0.873      -0.040       0.034
임대주택자산_요약_              0.3487    525.658      0.001      0.999   -1029.923    1030.620
비유동자산_요약_              -0.0907      0.124     -0.731      0.465      -0.334       0.153
투자자산_요약_                0.1892      0.088      2.153      0.031       0.017       0.361
_장기금융상품_요약_             0.0005      0.000      3.645      0.000       0.000       0.001
_투자유가증권_장기투자증권__요약_  3.253e-06   5.03e-06      0.647      0.518    -6.6e-06    1.31e-05
유형자산_요약_               -0.0963      0.105     -0.916      0.360      -0.302       0.110
토지                      0.2855      0.077      3.713      0.000       0.135       0.436
건설중인자산                  0.2855      0.077      3.712      0.000       0.135       0.436
건물                      0.2855      0.077      3.712      0.000       0.135       0.436
기계장치                    0.2855      0.077      3.712      0.000       0.135       0.436
차량운반구                   0.2857      0.077      3.715      0.000       0.135       0.436
렌탈자산                    0.2856      0.077      3.713      0.000       0.135       0.436
설비자산                    0.2855      0.077      3.712      0.000       0.135       0.436
무형자산_요약_                0.1892      0.088      2.153      0.031       0.0

In [46]:
df

,자산총계_요약_,유동자산_요약_,당좌자산_요약_,현금_및_현금성자산_요약_,단기투자자산_요약_,단기예금_요약_,단기매매증권_요약_,단기대여금_요약_,기타단기투자자산_요약_,매출채권_요약_,...,PCR_Price_cash_flow_ratio__최저_,PSR_Price_sales_ratio__최고_,PSR_Price_sales_ratio__최저_,기업가치_EV_,EBITDA,EBITDA_매출액,EBITDA_금융비용,EBITDA_평균발행주식수,EV_EBITDA,부도
0,28990.0,21169.0,10541.0,4330.0,4552.0,4539.0,1.0,13.0,0.0,786.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
1,20908.0,16359.0,2307.0,1829.0,18.0,0.0,6.0,13.0,0.0,14.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
2,15401.0,9880.0,1436.0,355.0,246.0,240.0,6.0,0.0,0.0,26.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
3,11063.0,8534.0,1083.0,2.0,381.0,380.0,0.0,0.0,1.0,72.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
4,5179.0,3893.0,445.0,11.0,0.0,0.0,0.0,0.0,0.0,33.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,79137.0,14794.0,12274.0,1440.0,5065.0,0.0,0.0,0.0,5065.0,5158.0,...,35.62,0.43,0.28,55934.79,1348.52,0.94,10.71,89.90,41.48,1.0
27353,79912.0,15880.0,13124.0,1588.0,5058.0,0.0,0.0,0.0,5058.0,6315.0,...,24.31,0.34,0.24,46036.63,1147.56,0.74,9.02,76.50,40.12,1.0
27354,75206.0,12515.0,11029.0,4017.0,5.0,0.0,0.0,0.0,5.0,4403.0,...,7.82,0.40,0.24,36557.57,1451.79,0.87,16.83,96.79,25.18,1.0
27355,83661.0,22620.0,19701.0,2318.0,11472.0,0.0,0.0,0.0,11472.0,5161.0,...,51.31,0.86,0.25,78081.78,1009.25,0.67,507.93,67.28,77.37,1.0


In [49]:
X = df.iloc[:,:-1]
y = df['부도']

In [50]:
model = sm.Logit(endog=y, exog=X).fit()

NameError: name 'sm' is not defined